# RamDA-SeqQC_template_SE (nbconvert)

## Directory setting

In [1]:
### working Dir (QCdata parent dir : "output_" + Project_id)
workdir = "."

### RamDA-QC-pipeline Dir
scriptdir = "."

In [4]:
setwd(workdir)
getwd()

source(paste0(scriptdir, "/00_sampleQC_function_nbconvert.R"))
outdir = ""

[1] "/data/21_Nextflow_test/test_githubcode_SE_fixed/output_190527_nextflow_ramdaQC_SE_unstranded"

## Loading exclude samplenames (blacklist)

In [ ]:
exclude_samplelist_path = paste0(workdir, "/exclude_samplelist.txt")

if (file.exists(exclude_samplelist_path)){
  exclude_samplename = scan(exclude_samplelist_path, what=character(), sep="\n", blank.lines.skip = F)
  exclude_samplename = exclude_samplename[exclude_samplename != ""]
  exclude_samplename
} else {
  cat("exclude samplelist not found.")
  exclude_samplename = ""
}

## 1. read QC results files

### > fastQC results

In [15]:
fastqc_SE_001 = read.table(paste0(workdir, "/summary_fastQC_result.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
fastqc_SE_001 = trim_dataset(fastqc_SE_001)
head(fastqc_SE_001)

### > RSeQC results (readDistribution)

In [17]:
### unstranded
readDist_SE_001 = read.table(paste0(workdir, "/summary_RSeQC_ReadDist_results_SE_sort.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
readDist_SE_001 = trim_dataset(readDist_SE_001)
head(readDist_SE_001)

### > RSeQC results  (geneBC)

In [18]:
### unstranded
geneBC_SE_001 = read.table(paste0(workdir, "/summary_RSeQC_geneBC_results_SE_sort.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
geneBC_SE_001 = trim_dataset(geneBC_SE_001)
head(geneBC_SE_001)

### > RSeQC results  (infer_experiment)

In [20]:
infer_SE_001 = read.table(paste0(workdir, "/summary_RSeQC_inferexperiment_results_SE_sort.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
infer_SE_001 = trim_dataset(infer_SE_001)
head(infer_SE_001)

### > highsensitivity rrna QC

In [ ]:
highsensitivity_rrna_result_path = paste0(workdir, "/highsensitivity_rrnaQC_summary/featureCounts.hisat2_G.T.R_loose_mapped.summary_simple.txt")

if (file.exists(highsensitivity_rrna_result_path)){
  highsenst_rrna_SE_001 = read.table(highsensitivity_rrna_result_path, sep="\t", comment.char = "", header=T, stringsAsFactors=F)

  ### edit data table
  highsenst_rrna_SE_001 = trim_dataset(highsenst_rrna_SE_001)
  head(highsenst_rrna_SE_001)
} else {
  cat("highsensitivity rrnaQC results not found.")
}

### > featurecounts results

#### >> featurecounts mrna results

In [25]:
fcount_SE_mrna_Ts_001 = read.table(paste0(workdir, "/summary_featurecounts_results_gencode_mrna_transcript.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
fcount_SE_mrna_Ts_001 = trim_dataset(fcount_SE_mrna_Ts_001)
head(fcount_SE_mrna_Ts_001)

#### >> featurecounts mt results

In [22]:
fcount_SE_mt_Ts_001 = read.table(paste0(workdir, "/summary_featurecounts_results_gencode_mt_transcript.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
fcount_SE_mt_Ts_001 = trim_dataset(fcount_SE_mt_Ts_001)
head(fcount_SE_mt_Ts_001)

#### >> featurecounts rrna results

In [40]:
fcount_SE_rrna_Ts_001 = read.table(paste0(workdir, "/summary_featurecounts_results_gencode_rrna_transcript.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
fcount_SE_rrna_Ts_001 = trim_dataset(fcount_SE_rrna_Ts_001)
head(fcount_SE_rrna_Ts_001)

#### >> featurecounts mrna_gene results

In [23]:
fcount_SE_mrna_gene_001 = read.table(paste0(workdir, "/summary_featurecounts_results_gencode_mrna_gene.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

### edit data table
fcount_SE_mrna_gene_001 = trim_dataset(fcount_SE_mrna_gene_001)
head(fcount_SE_mrna_gene_001)

## 2. plotting QC results

### Fig1. FastQC results (all samples)

#### >> total-seq

In [ ]:
g1 = plot_totalseq(fastqc_SE_001, "fastq_totalseq", outdir)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

#### >> perGC

In [ ]:
g1 = plot_perGC(fastqc_SE_001, "fastq_perGC", outdir)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

### Fig2. Read Distribution (exclude blacklist)

In [ ]:
g1 = plot_readDist_summary(readDist_SE_001, "bam_readDistribution", outdir, nonplot=exclude_samplename)

options(repr.plot.width=6, repr.plot.height=7)
plot_grid(g1)

#### >> genome assigned rate

In [ ]:
g1 = plot_assignedgene_rate(fastqc_SE_001, readDist_SE_001, "fastq_genome_assigned_rate", outdir, 100, nonplot=exclude_samplename)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

### Fig3. Gene body coverage (exclude blacklist)

In [ ]:
g1 = plot_geneBodyCov_heatmap(geneBC_SE_001, "bam_geneBodyCoverage", "", outdir, nonplot=exclude_samplename)

options(repr.plot.width=5, repr.plot.height=4)
plot_grid(g1)

### Fig4. Inferexperiment (exclude blacklist)

In [ ]:
g1 = plot_bar_inferexp(infer_SE_001, "bam_inferexperiment", outdir, nonplot=exclude_samplename)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

### Fig5. Highsensitivity rrna mapping QC (exclude blacklist)

In [ ]:
if (file.exists(highsensitivity_rrna_result_path)){
    
    g1 = plot_bar_highsenst_rrna(highsenst_rrna_SE_001, "bam_highsensitivity_rrnaQC", outdir, nonplot=exclude_samplename)

    options(repr.plot.width=10, repr.plot.height=4)
    plot_grid (g1)   
} else {
  cat("highsensitivity rrnaQC results not found.")
}

### Fig6. featurecounts (exclude blacklist)

#### >> mRNA, Ts

In [ ]:
g1 = plot_assigned_rate(fcount_SE_mrna_Ts_001, "featurecounts_mRNA_Ts", outdir, ylim=100, nonplot=exclude_samplename)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

#### >> mitocondria, Ts

In [ ]:
g1 = plot_assigned_rate(fcount_SE_mt_Ts_001, "featurecounts_mitocondria_Ts", outdir, ylim=100, nonplot=exclude_samplename)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

#### >> rRNA, Ts

In [ ]:
g1 = plot_assigned_rate(fcount_SE_rrna_Ts_001, "featurecounts_rRNA_Ts", outdir, ylim=100, nonplot=exclude_samplename)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

#### >> mRNA, Gene

In [ ]:
g1 = plot_assigned_rate(fcount_SE_mrna_gene_001, "featurecounts_mRNA_gene", outdir, ylim=100, nonplot=exclude_samplename)

options(repr.plot.width=10, repr.plot.height=4)
plot_grid (g1)

## 3. calc RPKM counts from featurecounts result

In [ ]:
fcount_SE_merge_001 = read.table(paste0(workdir, "/mergefcounts_gencode_mrna_gene.txt"), sep="\t", comment.char = "", header=T, stringsAsFactors=F)

dim(fcount_SE_merge_001)

In [ ]:
gene_length = data.frame(Geneid=fcount_SE_merge_001$Geneid, length=fcount_SE_merge_001$Length, stringsAsFactors=F)
head(gene_length)

### check "ERCC lengh" contains
dim(subset(gene_length, grepl("ERCC", Geneid)))

## 4. plotting detected genes num

In [ ]:
fcount_SE_merge_001_rpkm_set = calc_rpkm_counts(fcount_SE_merge_001, gene_length, exclude_name=exclude_samplename)

dim(fcount_SE_merge_001_rpkm_set$rpkm)
dim(fcount_SE_merge_001_rpkm_set$rpkm_log)

In [ ]:
fcount_SE_merge_001_rpkm_gene = fcount_SE_merge_001_rpkm_set$rpkm[!grepl("ERCC", rownames(fcount_SE_merge_001_rpkm_set$rpkm)),]
fcount_SE_merge_001_rpkm_gene_log = log10(fcount_SE_merge_001_rpkm_gene+1)

dim(fcount_SE_merge_001_rpkm_gene)
dim(fcount_SE_merge_001_rpkm_gene_log)

#### >> plot expression corr and detected genes

In [ ]:
### plot detgenes num
options(repr.plot.width=6, repr.plot.height=6)
bar_detgene_001 = plot_detgene_bar(fcount_SE_merge_001_rpkm_gene, fcount_SE_merge_001_rpkm_gene_log, nonplot=exclude_samplename)

In [ ]:
options(repr.plot.width=10, repr.plot.height=4)
plot_grid(bar_detgene_001)

## 5. plotting Dimensionality Reduction plot

In [ ]:
### pca
fcount_SE_dimentrd_set = create_pca_tsne_umap_mode(fcount_SE_merge_001_rpkm_gene_log, mode="pca", perplexity=10, local_connectivity=1.0, n_neighbors=15)
g1 = ggplot_2D(fcount_SE_dimentrd_set[["data_df"]], "PC1", "PC2", "pca plot", "", outdir)

options(repr.plot.width=5, repr.plot.height=4)
plot_grid(g1)

In [ ]:
### tsne
fcount_SE_dimentrd_set = create_pca_tsne_umap_mode(fcount_SE_merge_001_rpkm_gene_log, mode="tsne", perplexity=10, local_connectivity=1.0, n_neighbors=15)
g1 = ggplot_2D(fcount_SE_dimentrd_set[["data_df"]], "V1", "V2", "tsne plot", "", outdir)

options(repr.plot.width=5, repr.plot.height=4)
plot_grid(g1)

In [ ]:
### umap
fcount_SE_dimentrd_set = create_pca_tsne_umap_mode(fcount_SE_merge_001_rpkm_gene_log, mode="umap", perplexity=10, local_connectivity=1.0, n_neighbors=15)
g1 = ggplot_2D(fcount_SE_dimentrd_set[["data_df"]], "V1", "V2", "umap plot", "", outdir)

options(repr.plot.width=5, repr.plot.height=4)
plot_grid(g1)